# 任务3 特征工程&特征选择(3天)

# 特征工程

#核心代码举例

# 统计特征
    #计算均值
    gp = train.groupby(by)[fea].mean()
    #计算中位数
    gp = train.groupby(by)[fea].median()
    #计算方差
    gp = train.groupby(by)[fea].std()
    #计算最大值
    gp = train.groupby(by)[fea].max()
    #计算最小值
    gp = train.groupby(by)[fea].min()
    #计算出现次数
    gp = train.groupby(by)[fea].size()
    

# groupby生成统计特征：mean,std
    # 按照communityName分组计算面积的均值和方差
    temp = data.groupby('communityName')['area'].agg({'com_area_mean': 'mean', 'com_area_std': 'std'})

# 特征拆分
    # 将houseType转为'Room'，'Hall'，'Bath'
    def Room(x):
        Room = int(x.split('室')[0])
        return Room
    def Hall(x):
        Hall = int(x.split("室")[1].split("厅")[0])
        return Hall
    def Bath(x):
        Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
        return Bath

    data['Room'] = data['houseType'].apply(lambda x: Room(x))
    data['Hall'] = data['houseType'].apply(lambda x: Hall(x))
    data['Bath'] = data['houseType'].apply(lambda x: Bath(x))
    
#特征合并
    # 合并部分配套设施特征
    data['trainsportNum'] = 5 * data['subwayStationNum'] / data['subwayStationNum'].mean() + data['busStationNum'] / \
                                                                                             data[
                                                                                                 'busStationNum'].mean()

# 交叉生成特征:特征之间交叉+ - * / 
data['Room_Bath'] = (data['Bath']+1) / (data['Room']+1)


# 聚类特征
from sklearn.mixture import GaussianMixture  使用GaussianMixture做聚类特征
gmm = GaussianMixture(n_components=4, covariance_type='full', random_state=0)
gmm.fit_predict(data)
 
# 特征编码
from sklearn.preprocessing import LabelEncoder
data['communityName'] = LabelEncoder().fit_transform(data['communityName'])
from sklearn import preprocessing.OneHotEncoder
data['communityName'] = OneHotEncoder().fit_transform(data['communityName'])


# 过大量级值取log平滑（针对线性模型有效）
data[feature]=np.log1p(data[feature])

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('./data/train_data.csv')
test = pd.read_csv('./data/test_a.csv')
label = pd.read_csv('./data/评分文件/sub_a_913.csv')
target_train = train.pop('tradeMoney')


In [2]:
test = pd.concat([test, label], axis=1)
test.rename(columns={"pre":"tradeMoney"}, inplace=True)
target_test = test.pop('tradeMoney')
test.head()

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,tradeLandArea,landTotalPrice,landMeanPrice,totalWorkers,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime
0,100000079,36.54,未知方式,1室1厅1卫,高,21,西北,其他,XQ03867,SH,...,0.00,0,0.0,41503,319,301635,1601.0,397.0,2,2018/10/25
1,100000138,64.00,未知方式,2室1厅1卫,高,11,南北,其他,XQ00821,SH,...,0.00,0,0.0,4370,0,306857,21123.0,3066.0,0,2018/8/19
2,100000192,98.00,未知方式,3室1厅1卫,高,6,暂无数据,精装,XQ00842,SH,...,0.00,0,0.0,4370,0,306857,17471.0,2554.0,0,2018/3/4
3,100000200,43.45,未知方式,1室1厅1卫,高,6,暂无数据,其他,XQ00786,SH,...,0.00,0,0.0,4370,0,306857,19231.0,2016.0,0,2018/2/3
4,100000430,92.00,未知方式,2室2厅1卫,高,6,暂无数据,其他,XQ01201,SH,...,237828.35,1078790000,4536.0,20904,0,245872,26189.0,2957.0,0,2018/3/14


##  缺失值处理

In [3]:
def process_NAN(data):
    data['pv'].fillna(data['pv'].mean(), inplace=True)
    data['uv'].fillna(data['uv'].mean(), inplace=True)
    data['pv'] = data['pv'].astype('int')
    data['uv'] = data['uv'].astype('int')
    return data

train = process_NAN(train)
test = process_NAN(test)

In [4]:
# 对buildYear做处理
buildYearMean = pd.DataFrame(train[train['buildYear']!='暂无信息']['buildYear'].mode())
train.loc[train[train['buildYear'] == '暂无信息'].index, 'buildYear'] = buildYearMean.iloc[0,0]
train['buildYear'] = train['buildYear'].astype('int')

buildYearMean = pd.DataFrame(test[test['buildYear']!='暂无信息']['buildYear'].mode())
test.loc[test[test['buildYear'] == '暂无信息'].index, 'buildYear'] = buildYearMean.iloc[0,0]
test['buildYear'] = test['buildYear'].astype('int')

In [5]:
# 将houseType转为'Room'，'Hall'，'Bath'
def Room(x):
    Room = int(x.split('室')[0])
    return Room
def Hall(x):
    Hall = int(x.split("室")[1].split("厅")[0])
    return Hall
def Bath(x):
    Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
    return Bath

train['Room'] = train['houseType'].apply(lambda x: Room(x))
train['Hall'] = train['houseType'].apply(lambda x: Hall(x))
train['Bath'] = train['houseType'].apply(lambda x: Bath(x))
train['Room_Bath'] = (train['Bath']+1) / (train['Room']+1)

test['Room'] = test['houseType'].apply(lambda x: Room(x))
test['Hall'] = test['houseType'].apply(lambda x: Hall(x))
test['Bath'] = test['houseType'].apply(lambda x: Bath(x))
test['Room_Bath'] = (test['Bath']+1) / (test['Room']+1)

train.drop('houseType', axis=1, inplace=True)
test.drop('houseType', axis=1, inplace=True)

In [6]:
 def process_rentType(data):
    # 填充租房类型
    data.loc[(data['rentType'] == '未知方式') & (data['Room'] <=1), 'rentType'] = '整租'
    data.loc[(data['rentType'] == '未知方式') & (data['Room_Bath'] <= 1), 'rentType'] = '合租' 
    data.loc[(data['rentType'] == '未知方式') & (data['area'] / data['Room'] <= 20), 'rentType'] = '合租'

    # 小于50平米的房子，如果超过1个房间，说明有隔断
    data.loc[(data['rentType'] == '未知方式') & (data['Room'] > 1) & (data['area'] <=50), 'rentType'] = '合租'

    data.loc[(data['rentType'] == '未知方式') & (data['area'] > 60) & (data['Room'] == 2), 'rentType'] = '整租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] > 60) & (data['Room'] == 3), 'rentType'] = '整租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] >= 100) & (data['Room'] > 3), 'rentType'] = '整租'

    # 如果人均占比超过30平米
    # data.loc[(data['rentType'] == '未知方式') & (data['area'] / data['Room'] >30 ), 'rentType'] = '整租'
    return data

train = process_rentType(train)
test = process_rentType(test)

In [7]:
a = train.loc[(train['area']) > 60]
aa = a.loc[a['rentType'] == '整租']

In [8]:
aa.sort_values(by='area')

,ID,area,rentType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,region,...,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,Room,Hall,Bath,Room_Bath
36366,100008824,60.02,整租,中,14,南,其他,XQ00936,SH,RG00002,...,0,491767,39739,6001,0,2018/7/7,1,1,1,1.000000
10235,100223411,60.03,整租,中,14,南,精装,XQ01751,SH,RG00004,...,413,165159,4347,915,11,2018/11/2,1,1,1,1.000000
1434,100080491,60.05,整租,中,5,西,精装,XQ01943,SH,RG00005,...,0,187939,30251,2375,0,2018/2/22,2,1,1,0.666667
11852,100196687,60.05,整租,中,6,南,其他,XQ01540,SH,RG00004,...,0,248647,55639,6711,0,2018/5/11,1,1,1,1.000000
12255,100190481,60.05,整租,低,6,南,其他,XQ01540,SH,RG00004,...,0,248647,48990,5474,0,2018/7/25,1,1,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16877,100121838,689.67,整租,低,3,南,精装,XQ03384,SH,RG00011,...,0,97196,7381,1148,0,2018/9/19,5,3,6,1.166667
570,100072180,742.00,整租,低,3,暂无数据,精装,XQ00508,SH,RG00002,...,0,190706,51331,7105,0,2018/5/17,4,3,5,1.200000
34084,100020705,798.86,整租,低,2,南,精装,XQ01187,SH,RG00002,...,0,306857,22356,6545,0,2018/9/21,5,3,6,1.166667
1473,100052443,831.73,整租,低,2,西,毛坯,XQ02102,SH,RG00005,...,0,215146,12583,1257,0,2018/4/9,4,3,5,1.200000


In [9]:
bb = a.loc[a['rentType'] == '合租']
bb

,ID,area,rentType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,region,...,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,Room,Hall,Bath,Room_Bath
0,100309852,68.06,合租,低,16,暂无数据,其他,XQ00051,SH,RG00001,...,614,111546,1124,284,0,2018/11/28,2,1,1,0.666667
1,100307942,125.55,合租,中,14,暂无数据,简装,XQ00130,SH,RG00002,...,148,157552,701,22,1,2018/12/16,3,2,2,0.750000
2,100307764,132.00,合租,低,32,暂无数据,其他,XQ00179,SH,RG00002,...,520,131744,57,20,1,2018/12/22,3,2,2,0.750000
4,100305262,129.00,合租,低,2,暂无数据,毛坯,XQ01257,SH,RG00003,...,117,125309,2038,480,0,2018/11/18,3,2,3,1.000000
5,100304268,223.35,合租,低,2,暂无数据,其他,XQ00457,SH,RG00002,...,44561,719428,4832,1304,0,2018/11/14,4,3,3,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41387,100006315,85.61,合租,中,11,北,其他,XQ01228,SH,RG00002,...,0,49330,2462,403,0,2018/6/18,2,1,1,0.666667
41399,100005114,88.99,合租,高,27,北,其他,XQ01118,SH,RG00002,...,0,161754,10930,1766,0,2018/4/12,2,2,1,0.666667
41419,100002718,94.21,合租,中,29,北,其他,XQ01177,SH,RG00002,...,0,306857,21484,2570,0,2018/4/9,2,1,1,0.666667
41427,100001114,88.99,合租,低,27,北,其他,XQ01118,SH,RG00002,...,0,161754,10930,1766,0,2018/4/11,2,2,1,0.666667


In [10]:
def month(x):
    month = int(x.split('/')[1])
    return month

# 分割交易时间
train['month'] = train['tradeTime'].apply(lambda x: month(x))
test['month'] = test['tradeTime'].apply(lambda x: month(x))

train.drop('tradeTime', axis=1, inplace=True)
test.drop('tradeTime', axis=1, inplace=True)

In [11]:
def process_some_fields(data):

    # 合并部分配套设施特征
    data['trainsportNum'] = 5 * data['subwayStationNum'] / data['subwayStationNum'].mean() + data['busStationNum'] / \
                                                                                             data['busStationNum'].mean()
    data['all_schoolNum'] = 2 * data['interSchoolNum'] / data['interSchoolNum'].mean() + data['schoolNum'] / data['schoolNum'].mean() \
                            + data['privateSchoolNum'] / data['privateSchoolNum'].mean()
    data['all_hospitalNum'] = 2 * data['hospitalNum'] / data['hospitalNum'].mean() + \
                              data['drugStoreNum'] / data['drugStoreNum'].mean()
    data['all_mall'] = data['mallNum'] / data['mallNum'].mean() + \
                       data['superMarketNum'] / data['superMarketNum'].mean()
    data['otherNum'] = data['gymNum'] / data['gymNum'].mean() + data['bankNum'] / data['bankNum'].mean() + \
                       data['shopNum'] / data['shopNum'].mean() + 2 * data['parkNum'] / data['parkNum'].mean()

    data.drop(['subwayStationNum', 'busStationNum',
               'interSchoolNum', 'schoolNum', 'privateSchoolNum',
               'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum'],
              axis=1, inplace=True)
     
    data["area"] = data["area"].astype(int)
    categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate','cluster']

    return data, categorical_feats

train, train_categorical_feats = process_some_fields(train)
test, test_categorical_feats = process_some_fields(test)

## 计算统计特征

In [15]:
# data = feature_count(data, ['communityName','totalFloor'])

In [12]:
#计算统计特征
def featureCount(train,test):
    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    def feature_count(data, features=[]):
        new_feature = 'count'
        for i in features:
            new_feature += '_' + i
        temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
        data = data.merge(temp, 'left', on=features)
        return data

    data = feature_count(data, ['communityName'])
    data = feature_count(data, ['buildYear'])
    data = feature_count(data, ['totalFloor'])
    data = feature_count(data, ['communityName', 'totalFloor'])
    data = feature_count(data, ['communityName', 'newWorkers'])
    data = feature_count(data, ['communityName', 'totalTradeMoney'])
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return new_train, new_test
    
train, test = featureCount(train, test)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41440 entries, 0 to 41439
Data columns (total 51 columns):
ID                                     41440 non-null int64
area                                   41440 non-null int32
rentType                               41440 non-null object
houseFloor                             41440 non-null object
totalFloor                             41440 non-null int64
houseToward                            41440 non-null object
houseDecoration                        41440 non-null object
communityName                          41440 non-null object
city                                   41440 non-null object
region                                 41440 non-null object
plate                                  41440 non-null object
buildYear                              41440 non-null int32
saleSecHouseNum                        41440 non-null int64
totalTradeMoney                        41440 non-null int64
totalTradeArea                         4144

## groupby方法生成统计特征

In [12]:
#groupby生成统计特征：mean,std等

def gourpby(train,test):
    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    columns = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate']
    for feature in columns:
        data[feature] = LabelEncoder().fit_transform(data[feature])

    temp = data.groupby('communityName')['area'].agg({'com_area_mean': 'mean', 'com_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
    
    data['price_per_area'] = data.tradeMeanPrice / data.area * 100
    temp = data.groupby('communityName')['price_per_area'].agg(
        {'comm_price_mean': 'mean', 'comm_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
   
    temp = data.groupby('plate')['price_per_area'].agg(
        {'plate_price_mean': 'mean', 'plate_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    data.drop('price_per_area', axis=1, inplace=True)

    temp = data.groupby('plate')['area'].agg({'plate_area_mean': 'mean', 'plate_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    
    temp = data.groupby(['plate'])['buildYear'].agg({'plate_year_mean': 'mean', 'plate_year_std': 'std'})
    data = data.merge(temp, on='plate', how='left')
    data.plate_year_mean = data.plate_year_mean.astype('int')
    data['comm_plate_year_diff'] = data.buildYear - data.plate_year_mean
    data.drop('plate_year_mean', axis=1, inplace=True)

    temp = data.groupby('plate')['trainsportNum'].agg('sum').reset_index(name='plate_trainsportNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['trainsportNum'].agg('sum').reset_index(name='com_trainsportNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['trainsportNum_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                           data['com_trainsportNum'], data['plate_trainsportNum']))
    data = data.drop(['com_trainsportNum', 'plate_trainsportNum'], axis=1)

    temp = data.groupby('plate')['all_schoolNum'].agg('sum').reset_index(name='plate_all_schoolNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['all_schoolNum'].agg('sum').reset_index(name='com_all_schoolNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data = data.drop(['com_all_schoolNum', 'plate_all_schoolNum'], axis=1)

    temp = data.groupby(['communityName', 'plate'])['all_mall'].agg('sum').reset_index(name='com_all_mall')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')

    temp = data.groupby('plate')['otherNum'].agg('sum').reset_index(name='plate_otherNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['otherNum'].agg('sum').reset_index(name='com_otherNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['other_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                   data['com_otherNum'], data['plate_otherNum']))
    data = data.drop(['com_otherNum', 'plate_otherNum'], axis=1)

    temp = data.groupby(['month', 'communityName']).size().reset_index(name='communityName_saleNum')
    data = data.merge(temp, on=['month', 'communityName'], how='left')
    temp = data.groupby(['month', 'plate']).size().reset_index(name='plate_saleNum')
    data = data.merge(temp, on=['month', 'plate'], how='left')

    data['sale_ratio'] = round((data.communityName_saleNum + 1) / (data.plate_saleNum + 1), 3)
    data['sale_newworker_differ'] = 3 * data.plate_saleNum - data.newWorkers
    data.drop(['communityName_saleNum', 'plate_saleNum'], axis=1, inplace=True)

    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return new_train, new_test

train, test = gourpby(train, test)

In [13]:
train = train.drop('city', axis=1)
test = test.drop('city', axis=1)

## 聚类方法

In [14]:
#聚类
def cluster(train,test):
    from sklearn.mixture import GaussianMixture

    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    col = ['totalFloor',
           'houseDecoration', 'communityName', 'region', 'plate', 'buildYear',

           'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
           'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',

           'landTotalPrice', 'landMeanPrice', 'totalWorkers',
           'newWorkers', 'residentPopulation', 'lookNum',
           'trainsportNum',
           'all_schoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']

    # EM
    gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    data['cluster']= pd.DataFrame(gmm.fit_predict(data[col]))


    col1 = ['totalFloor','houseDecoration', 'communityName', 'region', 'plate', 'buildYear']
    col2 = ['tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
            'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',
            'landTotalPrice', 'landMeanPrice', 'totalWorkers',
            'newWorkers', 'residentPopulation', 'lookNum',
            'trainsportNum',
            'all_schoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']
    for feature1 in col1:
        for feature2 in col2:
        
            temp = data.groupby(['cluster',feature1])[feature2].agg('mean').reset_index(name=feature2+'_'+feature1+'_cluster_mean')
            temp.fillna(0, inplace=True)
       
            data = data.merge(temp, on=['cluster', feature1], how='left')
    
   
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    
    return new_train, new_test

train, test = cluster(train, test)   

In [15]:
train.to_csv("./data/FE_Housing_train.csv", index=False, header=True)
test.to_csv("./data/FE_Housing_test.csv", index=False, header=True)

## log平滑

In [23]:
# 过大量级值取log平滑（针对线性模型有效）
big_num_cols = ['totalTradeMoney','totalTradeArea','tradeMeanPrice','totalNewTradeMoney', 'totalNewTradeArea',
                'tradeNewMeanPrice','remainNewNum', 'supplyNewNum', 'supplyLandArea',
                'tradeLandArea','landTotalPrice','landMeanPrice','totalWorkers','newWorkers',
                'residentPopulation','pv','uv']
for col in big_num_cols:
    train[col] = train[col].map(lambda x: np.log1p(x))
    test[col] = test[col].map(lambda x: np.log1p(x))
        

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41440 entries, 0 to 41439
Columns: 174 entries, ID to otherNum_buildYear_cluster_mean
dtypes: float64(140), int32(9), int64(25)
memory usage: 55.2 MB


In [36]:
#对比特征工程前后线性模型结果情况
test=test.fillna(0)
# Lasso回归
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(train)
y_pred_test=lasso.predict(test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# 特征选择

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
#读取数据
train = pd.read_csv('')
test = pd.read_csv('')

target_train = train.pop('tradeMoney')
target_test = test.pop('tradeMoney')

## 相关系数法

In [ ]:
#相关系数法特征选择
from sklearn.feature_selection import SelectKBest

print(train.shape)

sk=SelectKBest(k=150)
new_train=sk.fit_transform(train,target_train)
print(new_train.shape)

# 获取对应列索引
select_columns=sk.get_support(indices = True)
# print(select_columns)

# 获取对应列名
# print(test.columns[select_columns])
select_columns_name=test.columns[select_columns]
new_test=test[select_columns_name]
print(new_test.shape)
# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

## Wrapper

In [ ]:
# Wrapper

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
rfe = RFE(lr, n_features_to_select=160)
rfe.fit(train,target_train)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False),
    n_features_to_select=40, step=1, verbose=0)




select_columns = [f for f, s in zip(train.columns, rfe.support_) if s]
print(select_columns)
new_train = train[select_columns]
new_test = test[select_columns]

# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

## Embedded
### 基于惩罚项的特征选择法
### Lasso(l1)和Ridge(l2)

In [ ]:
# Embedded
# 基于惩罚项的特征选择法
# Lasso(l1)和Ridge(l2)

from sklearn.linear_model import Ridge

ridge = Ridge(alpha=5)
ridge.fit(train,target_train)

Ridge(alpha=5, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

# 特征系数排序
coefSort = ridge.coef_.argsort()
print(coefSort)


# 特征系数
featureCoefSore=ridge.coef_[coefSort]
print(featureCoefSore)


select_columns = [f for f, s in zip(train.columns, featureCoefSore) if abs(s)> 0.0000005 ] 
# 选择绝对值大于0.0000005的特征

new_train = train[select_columns]
new_test = test[select_columns]
# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

### 基于树模型的特征选择法
### 随机森林 平均不纯度减少（mean decrease impurity

In [ ]:
# Embedded
# 基于树模型的特征选择法
# 随机森林 平均不纯度减少（mean decrease impurity


from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# 训练随机森林模型，并通过feature_importances_属性获取每个特征的重要性分数。rf = RandomForestRegressor()
rf.fit(train,target_train)
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), train.columns),
             reverse=True))

select_columns = [f for f, s in zip(train.columns, rf.feature_importances_) if abs(s)> 0.00005 ] 
# 选择绝对值大于0.00005的特征

new_train = train[select_columns]
new_test = test[select_columns]

# Lasso回归
from sklearn.linear_model import Lasso

lasso=Lasso(alpha=0.1)
lasso.fit(new_train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(new_train)

y_pred_test=lasso.predict(new_test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)